<a href="https://colab.research.google.com/github/quetion/Basic-Python-Programming/blob/master/VIX_construct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

# output setting
plt.style.use('seaborn-white')
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)
pd.options.display.width = 130

<ipython-input-1-d11939743521>:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
# 設定雲端硬碟中資料夾位置
os.chdir('/content/drive/MyDrive/[0]博士班/112_2/期貨與選擇權_台大/VIX white paper/data')
!ls -l #列出當下資料夾的檔案

total 133615
-rw------- 1 root root     17841 Apr 22 16:36 daily-treasury-rates_2019.csv
-rw------- 1 root root    624582 Apr 24 06:15 IvyDB_US_v5.4_Reference_Manual.pdf
-rw------- 1 root root     39933 Apr 24 08:29 rf_term.zip
-rw------- 1 root root 133525356 Apr 22 16:18 spx_2019.zip
-rw------- 1 root root   2126794 Apr 23 08:03 spx_202302_test.zip
-rw------- 1 root root    112662 Apr 24 09:13 VIX_construct.ipynb
-rw------- 1 root root    372868 Mar 28 05:33 vixts.csv


In [ ]:
# import zip-data
dfoptions = pd.read_csv('spx_2019.zip',compression='zip',
                        usecols = ['date','symbol','exdate','cp_flag','strike_price',
                                   'best_bid','best_offer','am_settlement','expiry_indicator'])
# reset the variable
dfoptions['strike_price'] = dfoptions['strike_price']/1000
# reset the date format
dfoptions['date'] = pd.to_datetime(dfoptions['date'],format='%Y-%m-%d')
dfoptions['exdate'] = pd.to_datetime(dfoptions['exdate'],format='%Y-%m-%d')
# check the dfoptions information
#dfoptions.info()

In [ ]:
# import interest rate (from )
dfrf = pd.read_csv('daily-treasury-rates_2019.csv')
# rename one of columns
dfrf.rename(columns = {'Date':'date'}, inplace=True)
# reset the date format
dfrf['date'] = pd.to_datetime(dfrf['date'],format='%m/%d/%Y')
# set 'date' to be the index
dfrf = dfrf.sort_values('date').set_index('date')
dfrf = dfrf/100
# reset the index from 0 to n
dfrf = dfrf.reset_index()

In [ ]:
dfrf.head()

,date,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2019-01-02,0.0240,0.0240,0.0242,0.0251,0.0260,0.0250,0.0247,0.0249,0.0256,0.0266,0.0283,0.0297
1,2019-01-03,0.0242,0.0242,0.0241,0.0247,0.0250,0.0239,0.0235,0.0237,0.0244,0.0256,0.0275,0.0292
2,2019-01-04,0.0240,0.0242,0.0242,0.0251,0.0257,0.0250,0.0247,0.0249,0.0256,0.0267,0.0283,0.0298
3,2019-01-07,0.0242,0.0242,0.0245,0.0254,0.0258,0.0253,0.0251,0.0253,0.0260,0.0270,0.0286,0.0299
4,2019-01-08,0.0240,0.0242,0.0246,0.0254,0.0260,0.0258,0.0257,0.0258,0.0263,0.0273,0.0288,0.0300


In [ ]:
SPX = dfoptions[~dfoptions['symbol'].str.contains('SPXW')].copy()


In [ ]:
# SPX's exweekday could be "Friday" or "Thursday"
# In general , they are Friday
# In cases of the Friday being the holiday, the expiration could be Thursday
aa = SPX['exweekday'].unique()
SPX[SPX['exweekday'] == aa[1]].head()

,date,symbol,exdate,cp_flag,strike_price,best_bid,best_offer,am_settlement,expiry_indicator,exweekday
1864,2019-01-02,SPX 190418C1000000,2019-04-18,C,1000.0,1494.9,1500.9,1,NaN,Thursday
1865,2019-01-02,SPX 190418C1100000,2019-04-18,C,1100.0,1395.8,1401.8,1,NaN,Thursday
1866,2019-01-02,SPX 190418C1200000,2019-04-18,C,1200.0,1296.7,1302.7,1,NaN,Thursday
1867,2019-01-02,SPX 190418C1225000,2019-04-18,C,1225.0,1272.0,1277.9,1,NaN,Thursday
1868,2019-01-02,SPX 190418C1250000,2019-04-18,C,1250.0,1247.2,1253.2,1,NaN,Thursday


In [ ]:
# Determine the days of am and pm settlement
# am-settlement : 開盤就結算
# pm-settlement : 收盤才結算
dfoptions['days'] = np.where(dfoptions['am_settlement'] == 0,
                             dfoptions['exdate'] - dfoptions['date'],
                             dfoptions['exdate'] - dfoptions['date'] - pd.to_timedelta(1,'days'))
# Determine the weekday in the expiration
dfoptions['exweekday'] = dfoptions['exdate'].dt.day_name()
# Select end-of-week SPXW and copy to dfSPXW
dfSPXW = dfoptions[dfoptions['symbol'].str.contains('SPXW')].copy()
# Select the SPXW of (i) expiration on Friday and (ii) weekly option (expiring at end of week) and (iii) PM-settled
# using "expiry_indicator" to justify the expiration on end of week ('w') or month ('m')
dfSPXW = dfSPXW[(dfSPXW['exweekday'] == 'Friday')&(dfSPXW['expiry_indicator']=='w')&(dfSPXW['am_settlement'] == 0)].copy()
# Combine end-of-week SPXWs with SPXs
dfoptions2 = pd.concat([dfSPXW, dfoptions[~dfoptions['symbol'].str.contains('SPXW')]]).sort_values(['date','days','cp_flag','strike_price'])
dfoptions2.head()

,date,symbol,exdate,cp_flag,strike_price,best_bid,best_offer,am_settlement,expiry_indicator,days,exweekday
4489,2019-01-02,SPXW 190104C800000,2019-01-04,C,800.0,1703.0,1711.1,0,w,2 days,Friday
4490,2019-01-02,SPXW 190104C900000,2019-01-04,C,900.0,1602.8,1610.6,0,w,2 days,Friday
4244,2019-01-02,SPXW 190104C1000000,2019-01-04,C,1000.0,1502.8,1510.6,0,w,2 days,Friday
4245,2019-01-02,SPXW 190104C1050000,2019-01-04,C,1050.0,1452.9,1460.6,0,w,2 days,Friday
4246,2019-01-02,SPXW 190104C1100000,2019-01-04,C,1100.0,1402.8,1410.6,0,w,2 days,Friday


In [ ]:
# A function outputs the near-term and next-term dfs for a given df at a given date
def keepNearNextTermOptions(df):
  maturities = pd.to_timedelta(df['days'].unique(),'days')
  nearT = maturities[maturities <= pd.to_timedelta(30,'days')].max()
  nextT = maturities[maturities > nearT].min()
  df1 = df[df['days'] == nearT].copy()
  df2 = df[df['days'] == nextT].copy()
  return pd.concat([df1,df2])


In [ ]:
# Selected optinos are grouped by dates
dfgrouped = dfoptions2.groupby('date')
dfs = []
# For every date, find the near-term and next-term of maturities. Append them to a new df
for date,df in dfgrouped:
  dfnearnext = keepNearNextTermOptions(df)
  dfs.append(dfnearnext)
# the near-term and next-term options have been selelcted
dfoptions3 = pd.concat(dfs)

In [ ]:
dfrfoptions.head(10)

,date,days
9370,2019-01-02,30 days
10288,2019-01-02,37 days
24106,2019-01-03,29 days
25223,2019-01-03,36 days
39158,2019-01-04,28 days
40295,2019-01-04,35 days
53986,2019-01-07,25 days
55139,2019-01-07,32 days
68514,2019-01-08,24 days
69667,2019-01-08,31 days


In [ ]:
# A function applying the above method and the cubic spline method to calculate the riskfree rate
def Riskfree(df, dfrf=dfrf):
  t = df['daynum']
  date = df['date']
  yieldcurve = dfrf[dfrf['date'] == date]
  if t==30:
    BEY = yieldcurve['1 Mo'].squeeze()
  elif (t>30)&(t<60):
    CSinter = CubicSpline([30,60],[yieldcurve['1 Mo'].squeeze(),yieldcurve['2 Mo'].squeeze()],bc_type='natural')
    BEY = float(CSinter(t))
  elif t<30:
    x = 1
    z = 1
    CMT1 = yieldcurve['1 Mo'].squeeze()
    for i in np.arange(2, len(yieldcurve.columns),1):
      if yieldcurve.iloc[0,i]>=CMT1:
        CMTx = yieldcurve.iloc[0,i]
        x = i
        break
    for i in np.arange(2, len(yieldcurve.columns),1):
      if yieldcurve.iloc[0,i]<=CMT1:
        CMTz = yieldcurve.iloc[0,i]
        z = i
        break
    keys = np.arange(2,len(yieldcurve.columns),1)
    values= [60,91,182,365,730,1095,1825,2555,3650,7300,10950]
    d = dict(zip(keys,values))
    if x == 1:
      blower=CMT1
    else:
      blower=CMT1 - ((CMTx-CMT1)/(d[x]-30))*30
    if z == 1:
      bupper= CMT1
    else:
      bupper=CMT1-((CMTz-CMT1)/(d[z]-30))*30
    CSinter = CubicSpline([0,30],[blower,bupper],bc_type='natural')
    BEY=float(CSinter(t))

  APY =((1+BEY/2)**2)-1
  rf = np.log(1+APY)
  return rf



In [ ]:
Riskfree(dfrfoptions)

,date,days,daynum
9370,2019-01-02,30 days,30
10288,2019-01-02,37 days,37
24106,2019-01-03,29 days,29
25223,2019-01-03,36 days,36
39158,2019-01-04,28 days,28


In [ ]:
dfrfoptions.tail()

,date,days,daynum
3528255,2019-12-27,42 days,42
3542886,2019-12-30,25 days,25
3544488,2019-12-30,39 days,39
3558855,2019-12-31,24 days,24
3560701,2019-12-31,38 days,38


In [ ]:
dfrfoptions['daynum'] = dfrfoptions['days'].dt.days

In [ ]:
# Calculate the riskfree rates of near-term and next-term maturities of each dates
# extract the 'date','days' columns and drop the duplicates
dfrfoptions = dfoptions3[['date','days']].drop_duplicates().copy()
# tranfer 30 days to 30 (float)
dfrfoptions['daynum'] = dfrfoptions['days'].dt.days

In [ ]:
Riskfree()

TypeError: Riskfree() missing 1 required positional argument: 'df'

In [ ]:
dfrfoptions['rf'] = dfrfoptions.apply(Riskfree,axis=1)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:



# Merge with option data
dfoptions4 = dfoptions3.merge(dfrfoptions,on=['date','days'],how='outer')
dfoptions4.head()

NameError: name 'columns' is not defined

In [ ]:
# check the near- and next-term maturities
test['days'].unique()

<TimedeltaArray>
['30 days', '37 days']
Length: 2, dtype: timedelta64[ns]